### Community detection: Triangle Counting and Clustering Coefficient

Triangle counting is a community detection graph algorithm which is used to determine the number of triangles passing through each vertex in the graph data set. A vertex is part of a triangle when it has two adjacent vertices with an edge between. The triangle is a three-node subgraph, where every two nodes are connected. This algorithm returns a Graph object and we extract vertices from this triangle counting graph.

Triangle counting is used heavily in social network analysis. It provides a measure of clustering in the graph data which is useful for finding communities and measuring the cohesiveness of local communities in social network websites like LinkedIn or Facebook. 

Clustering Coefficient, an important metric in a social network, shows how much community around one node is tightly connected. The notion of clustering coefficient is inspired by this observation, and is the standard method of summarizing triangle counts. It is well known that some networks, especially social networks, have much higher clustering coefficients than random networks.

#### Network Average Clustering Coefficient

The clustering coefficient, along with the mean shortest path, can indicate a "small-world" effect. For the clustering coefficient to be meaningful it should be significantly higher than in version of the network where all of the edges have been shuffled.

The neighborhood of a node, u, is the set of nodes that are connected to u. If every node in the neighborhood of u is connected to every other node in the neighborhood of u, then the neighborhood of u is complete and will have a clustering coefficient of 1. If no nodes in the neighborhood of u are connected, then the clustering coefficient will be 0.

In [15]:
#Loading library
import py2neo as py2neo

In [16]:
#Accessing local Neo4j Server
py2neo.authenticate("localhost:7474", "neo4j", "neo4j")

In [17]:
graph = py2neo.Graph("http://localhost:7474/db/data/")

In [20]:
#Querying triangle number for each node
query = """
CALL algo.triangleCount.stream('Person', 'HAS_CONTACT', {concurrency:100})
YIELD nodeId, triangles;
"""

In [21]:
#Running query 
results = graph.data(query)

In [6]:
#Printing 10 nodes
results[:10]

[{'nodeId': 1401, 'triangles': 88},
 {'nodeId': 1402, 'triangles': 49},
 {'nodeId': 1403, 'triangles': 50},
 {'nodeId': 1404, 'triangles': 56},
 {'nodeId': 1405, 'triangles': 95},
 {'nodeId': 1406, 'triangles': 79},
 {'nodeId': 1407, 'triangles': 7},
 {'nodeId': 1408, 'triangles': 29},
 {'nodeId': 1412, 'triangles': 138},
 {'nodeId': 1413, 'triangles': 56}]

In [7]:
#Sorting dictionary containing list
from operator import itemgetter
newlist = sorted(results, key=itemgetter('triangles'),reverse=True) 

In [8]:
#Printing top 10 triangle nodes
newlist[:10]

[{'nodeId': 1454, 'triangles': 165},
 {'nodeId': 1435, 'triangles': 139},
 {'nodeId': 1412, 'triangles': 138},
 {'nodeId': 1427, 'triangles': 137},
 {'nodeId': 1486, 'triangles': 128},
 {'nodeId': 1498, 'triangles': 125},
 {'nodeId': 1431, 'triangles': 123},
 {'nodeId': 1484, 'triangles': 114},
 {'nodeId': 1453, 'triangles': 108},
 {'nodeId': 1468, 'triangles': 107}]

In [9]:
#Creating nodeid array
top_10 = []
for a in range(10):
    top_10.append(list(newlist[:10][a].values())[0])

In [10]:
#algo.triangleCount.stream returns nodeid's. In order to get person names of this nodes we need to query nodeids
query = """
MATCH (person:Person) 
WHERE ID(person) in ["""+str(top_10).strip('[]')+"""] 
RETURN person.name
"""

In [11]:
graph.data(query)

[{'person.name': 'Charlize Theron'},
 {'person.name': 'Rob Reiner'},
 {'person.name': 'Val Kilmer'},
 {'person.name': 'Tony Scott'},
 {'person.name': 'Helen Hunt'},
 {'person.name': 'Greg Kinnear'},
 {'person.name': 'Parker Posey'},
 {'person.name': 'Bruno Kirby'},
 {'person.name': 'Liv Tyler'},
 {'person.name': 'Richard Harris'}]

In [12]:
#Computing cluster coefficient
query = """
CALL algo.triangleCount('Person', 'HAS_CONTACT',
{concurrency:4, write:true, writeProperty:'triangles',clusteringCoefficientProperty:'coefficient'})
YIELD loadMillis, computeMillis, writeMillis, nodeCount, triangleCount, averageClusteringCoefficient;
"""

In [13]:
results = graph.data(query)

In [14]:
results

[{'averageClusteringCoefficient': 0.19542368158538823,
  'computeMillis': 11,
  'loadMillis': 5,
  'nodeCount': 133,
  'triangleCount': 2442,
  'writeMillis': 8}]